In [ ]:
%tensorflow_version 1.x
%matplotlib inline

import random
import math
import time
import re
import os
import shutil
import datetime
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

!cp -u /content/drive/MyDrive/sd_cloud_playground/cyberset_riko/cyberset_tf_64.zip .
!unzip -nq ./cyberset_tf_64.zip
!mkdir -p models_tf/last
!mkdir -p models_tf/best

In [ ]:
def show_image(image, label='', color=True):
    picture = image.copy()
    plt.figure(figsize=(10,5))
    plt.axis('off')
    if color:
        cv2.putText(picture, label, (0,25), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,0), 6, cv2.LINE_AA)
        cv2.putText(picture, label, (0,25), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,0), 2, cv2.LINE_AA)
        plt.imshow(picture)
    else:
        cv2.putText(picture, label, (0,25), cv2.FONT_HERSHEY_SIMPLEX, 1,(0), 6, cv2.LINE_AA)
        cv2.putText(picture, label, (0,25), cv2.FONT_HERSHEY_SIMPLEX, 1,(255), 2, cv2.LINE_AA)
        plt.imshow(picture, cmap='gray')
    plt.show()

def load_dataset(directory):
    images = ['{}/images/{}'.format(directory, x) for x in os.listdir('{}/images'.format(directory))]
    images.sort()
    with open('{}/labels.txt'.format(directory), 'r') as f:
        labels = f.read().splitlines()
    return images, labels

def get_labels_encoder(labels):
    reduced = list(set(labels))
    reduced.sort()
    encoder_dictionary = {}
    decoder_dictionary = {}
    for i, l in enumerate(reduced):
        code = [0.0 for j in range(0, i)] + [1.0] + [0.0 for j in range(i, len(reduced)-1)]
        encoder_dictionary[l] = code
        decoder_dictionary[np.argmax(code, axis=-1)] = l
    def encoder(label):
        return encoder_dictionary[label]
    def decoder(node):
        return decoder_dictionary[node]
    return encoder, decoder, len(encoder_dictionary)

In [ ]:
TRAIN_DIRECTORY = './cyberset_tf_64/train'
VALIDATION_DIRECTORY = './cyberset_tf_64/validation'
TEST_DIRECTORY = './cyberset_tf_64/test'
# MODELS_DIRECTORY = '/content/drive/MyDrive/sd_cloud_playground/cyberset_riko/models_tf'
MODELS_DIRECTORY = './models_tf'
BATCH_SIZE = 163
# LEARNING_RATE = 1e-3
LEARNING_RATE = 0.0001
# TRAINING_STEPS = 54000
# TRAINING_STEPS = 10000
TRAINING_STEPS = 30000
VALIDATION_STEP = 2000
IMAGE_SIZE = 64
CHANNELS = 3
# BILATERAL_SIZE = 9
# BILATERAL_SIGMA = 75

train_images, train_labels = load_dataset(TRAIN_DIRECTORY)
validation_images, validation_labels = load_dataset(VALIDATION_DIRECTORY)
test_images, test_labels = load_dataset(TEST_DIRECTORY)
labels_encoder, labels_decoder, labels_count = get_labels_encoder(train_labels + validation_labels + test_labels)
last_model = '{}/last/last'.format(MODELS_DIRECTORY)
best_model = '{}/best/best'.format(MODELS_DIRECTORY)

In [ ]:
def prepare_image(path, color=True):
    if color:
        image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    else:
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    # image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    # image = cv2.bilateralFilter(image, BILATERAL_SIZE, BILATERAL_SIGMA, BILATERAL_SIGMA)
    return image

def get_train_batch(images_list, labels_list, labels_encoder):
    images = list()
    labels = list()
    for i in range(BATCH_SIZE):
        index = random.randint(0, len(labels_list)-1)
        if CHANNELS > 1:
            image = prepare_image(images_list[index]) / 255.0
        else:
            image = np.expand_dims(prepare_image(images_list[index], color=False)/255.0, axis=-1)
        label = labels_encoder(labels_list[index])
        images.append(image)
        labels.append(label)
    return (np.array(images), np.array(labels))

def get_validation_batch(images_list, labels_list, labels_encoder, offset):
    images = list()
    labels = list()
    for i in range(offset, offset+BATCH_SIZE):
        index = random.randint(0, len(labels_list)-1)
        if CHANNELS > 1:
            image = prepare_image(images_list[index]) / 255.0
        else:
            image = np.expand_dims(prepare_image(images_list[index], color=False)/255.0, axis=-1)
        label = labels_encoder(labels_list[index])
        images.append(image)
        labels.append(label)
    return (np.array(images), np.array(labels))

def get_test_batch(images_list, labels_list, labels_encoder, index):
    images = list()
    labels = list()
    if CHANNELS > 1:
        image = prepare_image(images_list[index]) / 255.0
    else:
        image = np.expand_dims(prepare_image(images_list[index], color=False)/255.0, axis=-1)
    label = labels_encoder(labels_list[index])
    images.append(image)
    labels.append(label)
    return (np.array(images), np.array(labels))

In [ ]:
def convolve_2d(input_node, kernel_shape, bias_shape, stride=1, padding='VALID', name='convolution'):
    with tf.variable_scope(name):
        weights = tf.get_variable('weights', kernel_shape, initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float32)
        biases = tf.get_variable('biases', bias_shape, initializer=tf.truncated_normal_initializer(), dtype=tf.float32)
        result = tf.nn.conv2d(input_node, weights, strides=[1,stride,stride,1], padding=padding)
        output = tf.nn.bias_add(result, biases)
    return output

# 128x128x3
'''def build_network(input, labels_count, name='building',
                  c1_kernel=5, c1_layers=6, p1_size=2, p1_stride=2,
                  c2_kernel=5, c2_layers=16, p2_size=2, p2_stride=2,
                  f1_neurons=120, f2_neurons=84):
    with tf.variable_scope(name):
        batch = input.get_shape().as_list()[0]
        conv1 = convolve_2d(input, kernel_shape=[c1_kernel,c1_kernel,CHANNELS,c1_layers], bias_shape=[c1_layers], name='conv1')
        norm1 = tf.contrib.layers.batch_norm(conv1)
        act1 =  tf.nn.relu(norm1, name='act1')
        pool1 = tf.nn.max_pool(act1, ksize=[1,p1_size,p1_size,1], strides=[1,p1_stride,p1_stride,1], padding='VALID', name='pool1')
        conv2 = convolve_2d(pool1, kernel_shape=[c2_kernel,c2_kernel,c1_layers,c2_layers], bias_shape=[c2_layers], name='conv2')
        norm2 = tf.contrib.layers.batch_norm(conv2)
        act2 =  tf.nn.relu(norm2, name='act2')
        pool2 = tf.nn.max_pool(act2,ksize=[1,p2_size,p2_size,1], strides=[1,p2_stride,p2_stride,1], padding='VALID', name='pool2')
        reshap = tf.reshape(pool2,[batch, -1])
        full1 = tf.contrib.layers.fully_connected(reshap, num_outputs=f1_neurons, activation_fn=tf.nn.relu)
        full2 = tf.contrib.layers.fully_connected(full1, num_outputs=f1_neurons, activation_fn=tf.nn.relu)
        output = tf.contrib.layers.fully_connected(full2, num_outputs=labels_count, activation_fn=None)
        prediction = tf.squeeze(tf.argmax(output, axis=-1))
    return output, prediction'''

# 56x56 batch 200
# 32 64 128 nodes
# filter 5x5x1 stride 1
# maxpool 2x2 stride 1
# 1024 fc, softmax
# relu, lr 0.0001
# dropout keep 0.5
# 30 000 iterations with adam
'''def build_network(input, labels_count, name='building',
                  c1_kernel=5, c1_layers=32, p1_size=2, p1_stride=1,
                  c2_kernel=5, c2_layers=64, p2_size=2, p2_stride=1,
                  c3_kernel=5, c3_layers=128, p3_size=2, p3_stride=1,
                  f1_neurons=1024):
    with tf.variable_scope(name):
        batch = input.get_shape().as_list()[0]
        conv1 = convolve_2d(input, kernel_shape=[c1_kernel,c1_kernel,CHANNELS,c1_layers], bias_shape=[c1_layers], name='conv1')
        act1 =  tf.nn.relu(conv1, name='act1')
        pool1 = tf.nn.max_pool(act1, ksize=[1,p1_size,p1_size,1], strides=[1,p1_stride,p1_stride,1], padding='VALID', name='pool1')
        conv2 = convolve_2d(pool1, kernel_shape=[c2_kernel,c2_kernel,c1_layers,c2_layers], bias_shape=[c2_layers], name='conv2')
        act2 =  tf.nn.relu(conv2, name='act2')
        pool2 = tf.nn.max_pool(act2,ksize=[1,p2_size,p2_size,1], strides=[1,p2_stride,p2_stride,1], padding='VALID', name='pool2')
        conv3 = convolve_2d(pool2, kernel_shape=[c3_kernel,c3_kernel,c2_layers,c3_layers], bias_shape=[c3_layers], name='conv3')
        act3 =  tf.nn.relu(conv3, name='act3')
        pool3 = tf.nn.max_pool(act3,ksize=[1,p3_size,p3_size,1], strides=[1,p3_stride,p3_stride,1], padding='VALID', name='pool3')
        reshap = tf.reshape(pool3,[batch, -1])
        full1 = tf.contrib.layers.fully_connected(reshap, num_outputs=f1_neurons, activation_fn=tf.nn.relu)
        drop1 = tf.contrib.layers.dropout(full1, keep_prob=0.5)
        output = tf.contrib.layers.fully_connected(drop1, num_outputs=labels_count, activation_fn=None)
        prediction = tf.squeeze(tf.argmax(output, axis=-1))
    return output, prediction'''

# 3 conv layers
# each w/ 2x2 max_pool non_overlapping
# each w/ 3x3 kernel
# feature maps 16 32 64
# convolution stride 1
# relu everywhere except last
# 2 full 256 neurons and softmax 14 (classes)
# RMSProp update rule
# covers 64x96
def build_network(input, labels_count, name='building',
                  c1_kernel=3, c1_layers=16, p1_size=2, p1_stride=2,
                  c2_kernel=3, c2_layers=32, p2_size=2, p2_stride=2,
                  c3_kernel=3, c3_layers=64, p3_size=2, p3_stride=2,
                  f1_neurons=256):
    with tf.variable_scope(name):
        batch = input.get_shape().as_list()[0]
        conv1 = convolve_2d(input, kernel_shape=[c1_kernel,c1_kernel,CHANNELS,c1_layers], bias_shape=[c1_layers], name='conv1')
        norm1 = tf.contrib.layers.batch_norm(conv1)
        act1 =  tf.nn.relu(norm1, name='act1')
        pool1 = tf.nn.max_pool(act1, ksize=[1,p1_size,p1_size,1], strides=[1,p1_stride,p1_stride,1], padding='VALID', name='pool1')
        conv2 = convolve_2d(pool1, kernel_shape=[c2_kernel,c2_kernel,c1_layers,c2_layers], bias_shape=[c2_layers], name='conv2')
        norm2 = tf.contrib.layers.batch_norm(conv2)
        act2 =  tf.nn.relu(norm2, name='act2')
        pool2 = tf.nn.max_pool(act2,ksize=[1,p2_size,p2_size,1], strides=[1,p2_stride,p2_stride,1], padding='VALID', name='pool2')
        conv3 = convolve_2d(pool2, kernel_shape=[c3_kernel,c3_kernel,c2_layers,c3_layers], bias_shape=[c3_layers], name='conv3')
        norm3 = tf.contrib.layers.batch_norm(conv3)
        act3 =  tf.nn.relu(norm3, name='act3')
        pool3 = tf.nn.max_pool(act3,ksize=[1,p3_size,p3_size,1], strides=[1,p3_stride,p3_stride,1], padding='VALID', name='pool3')
        reshap = tf.reshape(pool3,[batch, -1])
        full1 = tf.contrib.layers.fully_connected(reshap, num_outputs=f1_neurons, activation_fn=tf.nn.relu)
        drop1 = tf.contrib.layers.dropout(full1, keep_prob=0.5)
        output = tf.contrib.layers.fully_connected(drop1, num_outputs=labels_count, activation_fn=None)
        prediction = tf.squeeze(tf.argmax(output, axis=-1))
    return output, prediction

def loss_function(output, labels, name='loss'):
    with tf.variable_scope(name):
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=labels)
        loss = tf.reduce_mean(cross_entropy)
    return loss

In [ ]:
def train_network(train_images, train_labels, labels_count, validation_images, validation_labels, labels_encoder,
                  start_from=0, last_accuracy=0):
    tf.reset_default_graph()
    input = tf.placeholder(tf.float32, shape=[BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNELS], name='input')
    labels = tf.placeholder(tf.float32, shape=[BATCH_SIZE,labels_count], name='labels')
    output, prediction = build_network(input, labels_count)
    loss = loss_function(output, labels)
    optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
    optimization = optimizer.minimize(loss)
    saver = tf.train.Saver()
    params = 0
    for v in tf.trainable_variables():
        params += np.array(v.get_shape().as_list()).prod()
    print('TRAINING STARTED -> params: {}'.format(params))
    config = tf.ConfigProto(allow_soft_placement=True)
    start_time = time.time()
    best_accuracy = last_accuracy
    with tf.Session(config=config) as session:
        if start_from == 0:
            session.run(tf.global_variables_initializer())
            session.run(tf.local_variables_initializer())
        else:
            loader = tf.train.Saver()
            loader.restore(session, '{}/last/last'.format(MODELS_DIRECTORY))
        for s in range(start_from, TRAINING_STEPS):
            train_batch = get_train_batch(train_images, train_labels, labels_encoder)
            loss_value, _ = session.run([loss,optimization], feed_dict={input:train_batch[0], labels:train_batch[1]})
            elapsed_time = time.time() - start_time
            print_frequency = TRAINING_STEPS // 100 if TRAINING_STEPS // 100 >= 10 else TRAINING_STEPS // 10 if TRAINING_STEPS // 10 >= 1 else 1
            s_plus = s + 1
            if s_plus % print_frequency == 0:
                elapsed_delta = datetime.timedelta(seconds=round(elapsed_time))
                print('TRAINING -> step: {}/{} | loss: {:.3} | elapsed: {}'.format(s_plus, TRAINING_STEPS, loss_value, elapsed_delta))
                saver.save(session, '{}/last/last'.format(MODELS_DIRECTORY))
                with open('{}/checkpoint_last.txt'.format(MODELS_DIRECTORY), 'w') as c:
                    c.write('{}\n'.format(s))
            if VALIDATION_STEP > 0 and s_plus % VALIDATION_STEP == 0:
                good_predictions = 0
                for v in range(0, len(validation_labels), BATCH_SIZE):
                    validation_batch = get_validation_batch(validation_images, validation_labels, labels_encoder, v)
                    predicted_value, loss_value = session.run([prediction,loss], feed_dict={input:validation_batch[0], labels:validation_batch[1]})
                    ground_truths = np.argmax(validation_batch[1], axis=-1)
                    good_predictions += (predicted_value == ground_truths).sum()
                print('VALIDATION -> correct: {}/{} | loss: {:.3} | accuracy: {:.3}'.format(good_predictions, len(validation_labels),
                                                                                            loss_value, good_predictions/len(validation_labels)))
                if good_predictions >= best_accuracy:
                    best_accuracy = good_predictions
                    saver.save(session, '{}/best/best'.format(MODELS_DIRECTORY))
                    with open('{}/checkpoint_best.txt'.format(MODELS_DIRECTORY), 'w') as c:
                        c.write('{}\n'.format(best_accuracy))
    end_time = time.time() - start_time
    end_delta = datetime.timedelta(seconds=round(end_time))
    print('TRAINING ENDED -> elapsed: {}'.format(end_delta))

def resume_training(train_images, train_labels, labels_count, validation_images, validation_labels, labels_encoder):
    with open('{}/checkpoint_last.txt'.format(MODELS_DIRECTORY), 'r') as c:
        lines = c.read().splitlines()
    if len(lines) > 0 and lines[0] != '':
        step = int(lines[0]) + 1
        with open('{}/checkpoint_best.txt'.format(MODELS_DIRECTORY), 'r') as c:
            lines = c.read().splitlines()
        if len(lines) > 0 and lines[0] != '':
            accuracy = int(lines[0])
            print('RESUMING TRAINING -> last: {}/{} | best: {}/{}'.format(step, TRAINING_STEPS, accuracy, len(validation_labels)))
            train_network(train_images, train_labels, labels_count, validation_images, validation_labels, labels_encoder,
                          start_from=step, last_accuracy=accuracy)
        else:
            print('RESUMING TRAINING -> failed')
    else:
        print('RESUMING TRAINING -> failed')

def test_network(model, test_images, test_labels, labels_count, labels_encoder):
    tf.reset_default_graph()
    input = tf.placeholder(tf.float32, shape=[1,IMAGE_SIZE,IMAGE_SIZE,CHANNELS], name='input')
    _, prediction = build_network(input, labels_count)
    loader = tf.train.Saver()
    config = tf.ConfigProto(allow_soft_placement=True)
    correct_predictions = 0
    total_number = len(test_labels)
    with tf.Session(config=config) as session:
        loader.restore(session, model)
        for s in range(total_number):
            batch = get_test_batch(test_images, test_labels, labels_encoder, s)
            predicted_value = session.run(prediction, feed_dict={input:batch[0]})
            if np.squeeze(predicted_value) == np.argmax(batch[1], axis=-1):
                correct_predictions += 1
        error_rate = 100.0 * (total_number - correct_predictions) / total_number
        print('TESTING -> model: {} | correct: {}/{} | error: {:.1f}%'.format(model.split('/')[-1], correct_predictions, total_number, error_rate))

In [ ]:
def query_network(model, images, labels, labels_count, labels_encoder, labels_decoder):
    tf.reset_default_graph()
    input = tf.placeholder(tf.float32, shape=[1,IMAGE_SIZE,IMAGE_SIZE,CHANNELS], name='input')
    _, prediction = build_network(input, labels_count)
    loader = tf.train.Saver()
    config = tf.ConfigProto(allow_soft_placement=True)
    total_number = len(labels)
    with tf.Session(config=config) as session:
        loader.restore(session, model)
        for s in range(total_number):
            batch = get_test_batch(images, labels, labels_encoder, s)
            predicted_value = session.run(prediction, feed_dict={input:batch[0]})
            show_image(prepare_image(test_images[s]))
            if np.squeeze(predicted_value) == np.argmax(batch[1], axis=-1):
                answer = labels_decoder(np.squeeze(predicted_value))
                print("QUERY | ANSWER: http://nilf.it/{}".format(answer))
            else:
                answer = labels_decoder(np.squeeze(predicted_value))
                truth = labels_decoder(np.argmax(batch[1], axis=-1)[0])
                print("QUERY | ANSWER: http://nilf.it/{} | TRUTH: http://nilf.it/{}".format(answer, truth))

# query_network(best_model, test_images, test_labels, labels_count, labels_encoder, labels_decoder)

In [ ]:
train_network(train_images, train_labels, labels_count, validation_images, validation_labels, labels_encoder)
# resume_training(train_images, train_labels, labels_count, validation_images, validation_labels, labels_encoder)

In [ ]:
test_network(last_model, test_images, test_labels, labels_count, labels_encoder)
test_network(best_model, test_images, test_labels, labels_count, labels_encoder)